# Section 6: Building the Complete System
This notebook assembles password strength analysis, salted hashing, and TOTP-based two-factor authentication into a cohesive demo authentication system.

In [2]:
from __future__ import annotations

import math
import string
import time
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Optional

import bcrypt
import pyotp
import qrcode

## Reusable Security Helpers
Define password analysis, hashing utilities, and TOTP provisioning helpers that power the authentication system.

In [3]:
COMMON_PASSWORDS = {
    "password","123456","12345678","qwerty","abc123","letmein","iloveyou","admin","welcome","monkey",
}

@dataclass(frozen=True)
class PasswordReport:
    password: str
    length: int
    length_score: int
    character_sets: Dict[str, bool]
    variety_score: int
    entropy_bits: float
    is_common: bool
    total_score: int
    verdict: str

def analyze_password(password: str) -> PasswordReport:
    """Score the supplied password using length, variety, and entropy heuristics."""
    length = len(password)
    length_score = int(length >= 8) + int(length >= 12)

    character_sets = {
        "lowercase": any(ch in string.ascii_lowercase for ch in password),
        "uppercase": any(ch in string.ascii_uppercase for ch in password),
        "digits": any(ch in string.digits for ch in password),
        "symbols": any(ch in string.punctuation for ch in password),
    }

    variety_score = sum(character_sets.values())

    pool_size = 0
    pool_size += len(string.ascii_lowercase) if character_sets["lowercase"] else 0
    pool_size += len(string.ascii_uppercase) if character_sets["uppercase"] else 0
    pool_size += len(string.digits) if character_sets["digits"] else 0
    pool_size += len(string.punctuation) if character_sets["symbols"] else 0

    entropy_bits = 0.0
    if pool_size and length:
        entropy_bits = length * math.log2(pool_size)

    is_common = password.lower() in COMMON_PASSWORDS

    total_score = length_score + variety_score
    if is_common:
        total_score = max(0, total_score - 2)

    if total_score >= 6 and entropy_bits >= 60:
        verdict = "strong"
    elif total_score >= 4 and entropy_bits >= 40:
        verdict = "moderate"
    else:
        verdict = "weak"

    return PasswordReport(
        password=password,
        length=length,
        length_score=length_score,
        character_sets=character_sets,
        variety_score=variety_score,
        entropy_bits=round(entropy_bits, 2),
        is_common=is_common,
        total_score=total_score,
        verdict=verdict,
    )

def hash_password(password: str, rounds: int = 12) -> bytes:
    """Return a bcrypt hash containing its own salt."""
    return bcrypt.hashpw(password.encode("utf-8"), bcrypt.gensalt(rounds))

def verify_password(password: str, hashed: bytes) -> bool:
    """Validate a password against a stored bcrypt hash."""
    return bcrypt.checkpw(password.encode("utf-8"), hashed)

def provision_totp(account_name: str, issuer: str = "SecureAuthSystem") -> Dict[str, str]:
    """Create a TOTP secret, provisioning URI, and QR code image."""
    secret = pyotp.random_base32()
    totp = pyotp.TOTP(secret)
    provisioning_uri = totp.provisioning_uri(name=account_name, issuer_name=issuer)

    qr = qrcode.make(provisioning_uri)
    qr_path = Path(f"{account_name.replace('@', '_at_')}_totp.png")
    qr.save(qr_path)

    return {
        "secret": secret,
        "provisioning_uri": provisioning_uri,
        "qr_path": str(qr_path.resolve()),
    }

## AuthenticationSystem Implementation
The class below enforces strong passwords at registration, stores salted bcrypt hashes, and wires in TOTP secrets for second-factor checks.

In [4]:
class AuthenticationSystem:
    """Minimal user store that couples strong passwords with TOTP 2FA."""

    def __init__(self) -> None:
        self.users: Dict[str, Dict[str, str]] = {}

    def register_user(self, username: str, password: str) -> Dict[str, str]:
        if username in self.users:
            raise ValueError("User already exists")

        report = analyze_password(password)
        if report.verdict == "weak":
            raise ValueError("Password is too weak; choose a stronger one.")

        password_hash = hash_password(password).decode("utf-8")
        totp_data = provision_totp(username)

        self.users[username] = {
            "password_hash": password_hash,
            "totp_secret": totp_data["secret"],
            "provisioning_uri": totp_data["provisioning_uri"],
            "qr_path": totp_data["qr_path"],
            "password_verdict": report.verdict,
        }

        return {
            "username": username,
            "password_verdict": report.verdict,
            "provisioning_uri": totp_data["provisioning_uri"],
            "qr_path": totp_data["qr_path"],
        }

    def authenticate(self, username: str, password: str, totp_code: str, *, valid_window: int = 1) -> bool:
        user = self.users.get(username)
        if not user:
            return False

        if not verify_password(password, user["password_hash"].encode("utf-8")):
            return False

        totp = pyotp.TOTP(user["totp_secret"])
        return bool(totp.verify(totp_code, valid_window=valid_window))

    def get_user_record(self, username: str) -> Optional[Dict[str, str]]:
        user = self.users.get(username)
        if not user:
            return None
        return user.copy()

## Scenario Simulation
Register a sample user, inspect stored records, and demonstrate both successful and failed authentication attempts.

In [5]:
auth_system = AuthenticationSystem()

registration = auth_system.register_user("alice@example.com", "Sup3r$trongPass2025!")
print("Registration summary:")
for key, value in registration.items():
    print(f"  {key}: {value}")

stored_record = auth_system.get_user_record("alice@example.com") or {}
print("\nStored user record (mock database entry):")
for key, value in stored_record.items():
    if key == "totp_secret":
        masked = value[:4] + "..."
        print(f"  {key}: {masked}")
    else:
        print(f"  {key}: {value}")

totp = pyotp.TOTP(stored_record["totp_secret"])
current_code = totp.now()
print("\nGenerated TOTP code:", current_code)

success = auth_system.authenticate("alice@example.com", "Sup3r$trongPass2025!", current_code)
print("Authentication with correct credentials:", success)

bad_password = auth_system.authenticate("alice@example.com", "WrongPassword", current_code)
print("Authentication with wrong password:", bad_password)

wrong_totp = auth_system.authenticate("alice@example.com", "Sup3r$trongPass2025!", "000000")
print("Authentication with wrong TOTP code:", wrong_totp)

time.sleep(2)
expired_success = auth_system.authenticate("alice@example.com", "Sup3r$trongPass2025!", current_code)
print("Authentication after code expires (likely False):", expired_success)

Registration summary:
  username: alice@example.com
  password_verdict: strong
  provisioning_uri: otpauth://totp/SecureAuthSystem:alice%40example.com?secret=VHSKGMPFTQLOFSJWXPKYQ3UG54D7WVU4&issuer=SecureAuthSystem
  qr_path: C:\Users\Micha\OneDrive\My Time at Goldsmiths\Year 3\Networks and System Security\E-Portfolio of Evidence\Week 03\alice_at_example.com_totp.png

Stored user record (mock database entry):
  password_hash: $2b$12$/cetlMX8paXqo661maS5v.yO1VbHUkNhNL84hvfoMBxPWX7hPGhXu
  totp_secret: VHSK...
  provisioning_uri: otpauth://totp/SecureAuthSystem:alice%40example.com?secret=VHSKGMPFTQLOFSJWXPKYQ3UG54D7WVU4&issuer=SecureAuthSystem
  qr_path: C:\Users\Micha\OneDrive\My Time at Goldsmiths\Year 3\Networks and System Security\E-Portfolio of Evidence\Week 03\alice_at_example.com_totp.png
  password_verdict: strong

Generated TOTP code: 653049
Authentication with correct credentials: True
Authentication with correct credentials: True
Authentication with wrong password: False
Authe